In [1]:
from src.lexer import Lexer
from src.parser import Parser
from src.parser.utils import ast_print, asdict

In [2]:
primitives = ["none", "5", "4.69", "'hello world'", "true", "false", "x"]
for primitive in primitives:
    lexer = Lexer(primitive)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_primary()
    print(node)

NoneNode()
NumberNode(value=5, type=NumberType.INTEGER)
NumberNode(value=4.69, type=NumberType.FLOAT)
StringNode(value='hello world')
BooleanNode(value=True)
BooleanNode(value=False)
IdentifierNode(name='x')


---

In [3]:
lines = [
    "some_list[0]", "x[0][1]", "x[index]", # postfixes
    "not true", "-3", "+2.78", # unary operators
    "human.name", "node.value.type", # attributes
    "- human.friends[2].age", # mixed up
]
for code in lines:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_unary_or_primary()
    print(node)

IndexNode(collection=IdentifierNode(name='some_list'), index=NumberNode(value=0, type=NumberType.INTEGER))
IndexNode(collection=IndexNode(collection=IdentifierNode(name='x'), index=NumberNode(value=0, type=NumberType.INTEGER)), index=NumberNode(value=1, type=NumberType.INTEGER))
IndexNode(collection=IdentifierNode(name='x'), index=IdentifierNode(name='index'))
UnaryOpNode(operator=UnaryOperatorType.NOT, operand=BooleanNode(value=True))
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=NumberNode(value=3, type=NumberType.INTEGER))
UnaryOpNode(operator=UnaryOperatorType.POSITIVE, operand=NumberNode(value=2.78, type=NumberType.FLOAT))
AttributeNode(object=IdentifierNode(name='human'), attribute='name')
AttributeNode(object=AttributeNode(object=IdentifierNode(name='node'), attribute='value'), attribute='type')
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=AttributeNode(object=IndexNode(collection=AttributeNode(object=IdentifierNode(name='human'), attribute='friends'), ind

---

In [4]:
expressions = [
    "5", " - (- 5)",
    "[1, 2, 3]", "['a', 'b', 'c']",
    "{'a': 1, 'b': 2}",
    "1 + 2", "47 - 58 * 6", "(47 - 58) * 6", "index + 7 * (4 + divisor / 5 % 7) ** 2",
    "78 > 47", "78 >= 47", "78 < 47", "78 <= 47", "78 == 47", "78 != 47", "78 == 47 and 47 < 78", "78 == 47 or 47 < 78",
    "42 if universe.question == 'The answer to life, the universe and everything' else none", # ternary
]
for code in expressions:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_expression()
    print(node)

NumberNode(value=5, type=NumberType.INTEGER)
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=NumberNode(value=5, type=NumberType.INTEGER)))
ListNode(elements=[NumberNode(value=1, type=NumberType.INTEGER), NumberNode(value=2, type=NumberType.INTEGER), NumberNode(value=3, type=NumberType.INTEGER)])
ListNode(elements=[StringNode(value='a'), StringNode(value='b'), StringNode(value='c')])
DictionaryNode(pairs=[(StringNode(value='a'), NumberNode(value=1, type=NumberType.INTEGER)), (StringNode(value='b'), NumberNode(value=2, type=NumberType.INTEGER))])
BinaryOpNode(left=NumberNode(value=1, type=NumberType.INTEGER), operator=BinaryOperatorType.PLUS, right=NumberNode(value=2, type=NumberType.INTEGER))
BinaryOpNode(left=NumberNode(value=47, type=NumberType.INTEGER), operator=BinaryOperatorType.MINUS, right=BinaryOpNode(left=NumberNode(value=58, type=NumberType.INTEGER), operator=BinaryOperatorType.MUL, right=NumberNode(value=6, ty

---

In [5]:
assignments = [
    "x = 5",
    "x += velocity",
    "x -= velocity",
    "x *= velocity",
    "x /= velocity",
    "x %= velocity",
    "x **= velocity",
]
for code in assignments:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_statement()
    print(node)

AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.ASSIGN, value=NumberNode(value=5, type=NumberType.INTEGER))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.PLUS, value=IdentifierNode(name='velocity'))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.MINUS, value=IdentifierNode(name='velocity'))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.MUL, value=IdentifierNode(name='velocity'))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.DIV, value=IdentifierNode(name='velocity'))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.MOD, value=IdentifierNode(name='velocity'))
AssignmentNode(target=IdentifierNode(name='x'), operator=AssignmentOperatorType.POWER, value=IdentifierNode(name='velocity'))


---

In [6]:
from tests.parser.unit_tests import get_test_cases, run_tests
tests = get_test_cases()
run_tests(tests)

  Testing PARSE_PRIMARY
-------------------------------------
none
NoneNode()
-------------------------------------
5
NumberNode(value=5, type=NumberType.INTEGER)
-------------------------------------
4.69
NumberNode(value=4.69, type=NumberType.FLOAT)
-------------------------------------
'hello world'
StringNode(value='hello world')
-------------------------------------
true
BooleanNode(value=True)
-------------------------------------
false
BooleanNode(value=False)
-------------------------------------
x
IdentifierNode(name='x')

  Testing PARSE_UNARY_OR_PRIMARY
-------------------------------------
some_list[0]
IndexNode(collection=IdentifierNode(name='some_list'), index=NumberNode(value=0, type=NumberType.INTEGER))
-------------------------------------
x[0][1]
IndexNode(collection=IndexNode(collection=IdentifierNode(name='x'), index=NumberNode(value=0, type=NumberType.INTEGER)), index=NumberNode(value=1, type=NumberType.INTEGER))
-------------------------------------
x[index]
Index

In [7]:

blocks = [
    "if true { 45 }",
    "if true { 45 } else { 46 }",
    "if true 45",
    "while true { 45; 46 }",
    "x in [1, 2, 3]",
    "return", "return some_value",
    "for x in list { break }",
    "for x in list { continue }",
    "while cond { break }",
    "while cond { continue }",
]

for code in blocks:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_statement()
    print(node)

IfNode(condition=BooleanNode(value=True), then_block=[NumberNode(value=45, type=NumberType.INTEGER)], else_block=None)
IfNode(condition=BooleanNode(value=True), then_block=[NumberNode(value=45, type=NumberType.INTEGER)], else_block=[NumberNode(value=46, type=NumberType.INTEGER)])
IfNode(condition=BooleanNode(value=True), then_block=[NumberNode(value=45, type=NumberType.INTEGER)], else_block=None)
WhileNode(condition=BooleanNode(value=True), body=[NumberNode(value=45, type=NumberType.INTEGER), NumberNode(value=46, type=NumberType.INTEGER)])
BinaryOpNode(left=IdentifierNode(name='x'), operator=BinaryOperatorType.IN, right=ListNode(elements=[NumberNode(value=1, type=NumberType.INTEGER), NumberNode(value=2, type=NumberType.INTEGER), NumberNode(value=3, type=NumberType.INTEGER)]))
ReturnNode(value=None)
ReturnNode(value=IdentifierNode(name='some_value'))
ForNode(variable='x', iterable=IdentifierNode(name='list'), body=[BreakNode()])
ForNode(variable='x', iterable=IdentifierNode(name='list')

---

In [8]:
with open("tests/parser/parse.txt", "r") as f:
    code = f.read()
    
print(code)

// literals
none
5
4.69
'hello world'
true
false
some_variable

// unary operations
-7
+4.5
not true

// postfixes (indexing, attributes)
some_list[47]
other_list[pre_index][post_index]

human.name
node.value.type

// mixed up
- human.friends[2].age

// lists
[1, 2, 3]
['a', 'b', 'c']
[1, 2, 3, [4, 5, 6]]

// dictionaries
{'a': 1, 'b': 2}
{'a': 1, 'b': 2, 'c': {'d': 3, 'e': 4}}

// binary operations
1 + 2
47 - 58 * 6
(47 - 58) * 6
index + 7 * (4 + divisor / 5 % 7) ** 2
78 > 47
78 >= 47
78 < 47
78 <= 47
78 == 47
78 != 47
12 == 21 and 21 < 12
12 == 21 or 21 < 12

// ternary operations
42 if universe.question == 'The answer to life, the universe and everything' else none

// assignments
x = 42
position += velocity * dt
position -= velocity * dt
power *= 2
power /= 2
seed %= 17
length **= 2

// function calls
print('Hello, world!')
print('Hello, world!', end='\n')
f(a, b, c, x = 3)

// function definitions
function f(a, b, c, x = 0) {
    return a + b + c + x
}

// control flow

if age > 1

In [9]:
lexer = Lexer(code)
tokens = lexer.tokenize()

parser = Parser(tokens)
ast = parser.parse()
for stmt in ast.statements:
    print(stmt)

NoneNode()
NumberNode(value=5, type=NumberType.INTEGER)
NumberNode(value=4.69, type=NumberType.FLOAT)
StringNode(value='hello world')
BooleanNode(value=True)
BooleanNode(value=False)
BinaryOpNode(left=BinaryOpNode(left=IdentifierNode(name='some_variable'), operator=BinaryOperatorType.MINUS, right=NumberNode(value=7, type=NumberType.INTEGER)), operator=BinaryOperatorType.PLUS, right=NumberNode(value=4.5, type=NumberType.FLOAT))
UnaryOpNode(operator=UnaryOperatorType.NOT, operand=BooleanNode(value=True))
IndexNode(collection=IdentifierNode(name='some_list'), index=NumberNode(value=47, type=NumberType.INTEGER))
IndexNode(collection=IndexNode(collection=IdentifierNode(name='other_list'), index=IdentifierNode(name='pre_index')), index=IdentifierNode(name='post_index'))
AttributeNode(object=IdentifierNode(name='human'), attribute='name')
BinaryOpNode(left=AttributeNode(object=AttributeNode(object=IdentifierNode(name='node'), attribute='value'), attribute='type'), operator=BinaryOperatorType.